In [1]:
"""
Evan Turner
This is my testing program to convert NOAA NAM formatted files to the TXBLEND winds
for import into https://github.com/twdb/coastal-spill 

Eventually this code will get ingested to https://github.com/evanleeturner/nam-dataserver


This is for my own internal testing and devlopment purposes.

"""
import pandas as pd
import numpy as np
import os
import logging
import datetime
import functools
import tarfile
import tabulate
my_level = logging.DEBUG  #set to the level you want, eg. ERROR, WARNING, INFO, DEBUG
logger = logging.getLogger()
logging.basicConfig(level=my_level)
logger.setLevel(my_level)  #have to set the level twice - I don't know why this is, but it just is!

logger.error("Error level message")
logger.warning("Warning level message")
logger.info("Info level message")
logger.debug("Debug level message")


ERROR:root:Error level message
INFO:root:Info level message
DEBUG:root:Debug level message


In [2]:
#libraries copied from wind_calcs.py
#https://github.com/evanleeturner/nam-dataserver/blob/main/src/namdataserver/wind_calcs.py
import numpy as np

def wind_spddir_to_uv(wspd,wdir):
    """
    calculated the u and v wind components from wind speed and direction
    Input:
        wspd: wind speed
        wdir: wind direction
    Output:
        u: u wind component
        v: v wind component
    """

    rad = 4.0*np.arctan(1)/180.
    u = -wspd*np.sin(rad*wdir)
    v = -wspd*np.cos(rad*wdir)

    return u,v

def wind_uv_to_dir(U,V):
    """
    Calculates the wind direction from the u and v component of wind.
    Takes into account the wind direction coordinates is different than the
    trig unit circle coordinate. If the wind directin is 360 then returns zero
    (by %360)
    Inputs:
      U = west/east direction (wind from the west is positive, from the east is negative)
      V = south/noth direction (wind from the south is positive, from the north is negative)
    """
    WDIR= (270-np.rad2deg(np.arctan2(V,U)))%360
    return WDIR

def wind_uv_to_spd(U,V):
    """
    Calculates the wind speed from the u and v wind components
    Inputs:
      U = west/east direction (wind from the west is positive, from the east is negative)
      V = south/noth direction (wind from the south is positive, from the north is negative)
    """
    WSPD = np.sqrt(np.square(U)+np.square(V))
    return WSPD


# Below is used for calculing the angle between two wind vectors
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """
    Calcualates the angle between two wind vecotrs. Utilizes the cos equation:
                cos(theta) = (u dot v)/(magnitude(u) dot magnitude(v))
    Input:
        v1 = vector 1. A numpy array, list, or tuple with
             u in the first index and v in the second --> vector1 = [u1,v1]
        v2 = vector 2. A numpy array, list, or tuple with
             u in the first index and v in the second --> vector2 = [u2,v2]
    Output:
    Returns the angle in radians between vectors 'v1' and 'v2'::
            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    angle = np.arccos(np.dot(v1_u, v2_u))
    if np.isnan(angle):
        if (v1_u == v2_u).all():
            return np.rad2deg(0.0)
        else:
            return np.rad2deg(np.pi)
    return np.rad2deg(angle)

In [3]:
def Print_Winds_TXBLEND_FMT(bigframe, twdb_wind_list, outputfolder):
    """
    Function prints a large combined dataframe of wind data in the TXBLEND wndq format file.  
    This format was originally what was delivered to TWDB by the TAMU modeling group through a web interface
    
    The function converts a dataframe that looks like this:
    
    	station	Datetime	SPD_mph	DIR
    0	twdb000	2022-08-14 18:00:00	15.128582	102.088511
    1	twdb001	2022-08-14 18:00:00	14.335636	108.621309
    2	twdb002	2022-08-14 18:00:00	15.999258	85.801288
    
    
    Tp the ending file format with fixed with crazyness like this:

    :*******Wind Data (3Hourly) Time in GMT****
    **3-hourly winds from NAM  model for station  twdb135 28.248   -96.326
     50, number of days of this record
    2022 03 19 00   11.0  354.8
    2022 03 19 03    9.4   12.2
    2022 03 19 06   10.5   31.9
    2022 03 19 09   16.5   48.4
    2022 03 19 12   16.4   49.8
    2022 03 19 15   15.8   60.4
    2022 03 19 18   12.0   70.3
    
    To accomplish this, we use a library called tabulate (https://pypi.org/project/tabulate/), 
    various string formats, and pandas.
    
    """
    logging.debug("Entering Print_Winds_TXBLEND_FMT() with bigframe of len {}, {}, {}"
                  .format(len(bigframe),twdb_wind_list,outputfolder))
    logging.info("Writing wind data in TXBLEND format: STARTING")
    
    #error checking...
    isdir = os.path.isdir(outputfolder)
    if not isdir:
        logging.info("The specified output directory {} does not exist.  Trying to create it now...".format(outputfolder))
        try:
            os.mkdir(outputfolder)
            logging.info("Succesfully created {}".format(outputfolder))
        except OSError as err:
            logging.error("Could not create directory {}.  Quitting!  OS error: {0}".format(outputfolder,err))
    
    isfile = os.path.isfile(twdb_wind_list)
    if not isfile:
        logging.error("CRITICAL: Could not locate valid TWDB wind listing file specified at {}  Exiting."
                      .format(twdb_wind_list))
        return
        
        
    
    try: 
        listdf = pd.read_csv(twdb_wind_list)
        logging.debug("Openned twdb_wind_list for reading with {} entries".format(len(listdf)))
    except:
        logging.error("CRITICAL: Pandas had critical error trying to open our wind list file {}"
                      .format(twdb_wind_list))
        return
    
    #we expect to have a dataframe with these exact columns.  Otherwise lets quit out and respond why.
    expected_columns = ['station', 'Datetime', 'SPD_mph', 'DIR']  
    logging.debug("Running check on our df columns {} to expected {}".format(bigframe.columns,expected_columns))
        
    #some crazy lamba function testing to see if our columns are exactly the same.  I lifted this from:
    #https://www.digitalocean.com/community/tutorials/how-to-compare-two-lists-in-python
    if not functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,expected_columns,bigframe.columns), True): 
        logging.error("CRITICAL: Our dataframe columns are NOT what we expected.  What was passed in is {} \n We expected {}"
                      .format(bigframe.columns,expected_columns))
        return
    
    for i in bigframe['station'].unique():
        logging.debug("Processing station {}".format(i))
        tmp = bigframe.loc[(bigframe['station'] == i)].copy()
        logging.debug("Our sliced df has len {} and looks like {}".format(len(tmp),tmp.head(2)))
        tmp['STRTIME'] = tmp['Datetime'].dt.strftime('%Y %m %d %H')
        tmp['HOUR'] = tmp['Datetime'].dt.strftime('%H')
        
        #drop any duplicates - if they exist...
        tmp.drop_duplicates(subset=['Datetime'],inplace=True)
        
    
        #some crazy slicing here to get only the 3-hourly wind data...
        tmp  = tmp [(tmp['HOUR'] == '00') | (tmp['HOUR'] == '03')
                    | (tmp['HOUR'] == '06') | (tmp['HOUR'] == '09')
                   | (tmp['HOUR'] == '12') | (tmp['HOUR'] == '15')
                | (tmp['HOUR'] == '18') | (tmp['HOUR'] == '21')]
        #how many days are in the record??
        count = int(len(tmp)/8)
        
        tmp = tmp[['STRTIME', 'SPD_mph','DIR']]
        
        series = listdf.loc[listdf['station'] == i].squeeze()
        #need to separate the values from series to for python format print to work
        lon = series['lon']
        lat = series['lat']
        #for help with format printing, check out https://www.w3schools.com/python/ref_string_format.asp
        header_ln2 = "**3-hourly winds from NAM  model for station  {} {lat:.3f}   {lon:.3f}\n"
        header_ln3 = str(count)+ " number of days of this record\n"
        
        
        content = tabulate.tabulate(tmp.values.tolist(), tablefmt="plain", numalign="right",floatfmt=".1f",stralign="right")
        
        mypath = os.path.join(outputfolder,i)
        try:
            f = open(mypath+".wndq", "w")
        except:
            logging.error("CRITICAL: could not write file {} ".format(mypath+".wndq"))
            break              
        f.write("*******Wind Data (3Hourly) Time in GMT****\n")
        f.write(header_ln2.format(i,lat=lat,lon=lon))
        f.write(header_ln3)
        f.write(content)
        f.close()
        logging.debug("Writing file {} COMPLETED".format(mypath+".wndq"))
        
        
    logging.info("Print_Winds_TXBLEND_FMT() COMPLETE")
    return


In [4]:
def make_tarfile(output_filename, source_dir):
    """
    Method is a simple wrapper for tarfile library to create a tar.gz out of an entire directory.
    """
    logging.debug("Entering make_tarfile() with output {} and source {}".format(output_filename, source_dir))
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
    logging.info("Wrote tarfile {}".format(output_filename))

def read_TWDB_NAM_csv(fn,folder,columns_name, convertUVwinds=True):
    """
    Function reads a single file from disk in TWDB NAM CSV format.  
    
    An example file "nam_218_20220814_1800_000.csv" would look like:
    
    UGRD_P0_L103_GLC0,VGRD_P0_L103_GLC0,station
    -6.6131005,1.416339,twdb000
    -6.0731006,2.046339,twdb001
    -7.1331005,-0.5236609,twdb002
    -7.4331,1.036339,twdb003
    -6.7131004,1.936339,twdb004
    -5.8631005,2.2063391,twdb005
    -5.3731003,2.376339,twdb006
    -5.8531003,-1.2136608,twdb007
    -7.8431005,0.2863391,twdb008
    -8.1331005,1.8463391,twdb009
    -7.1031003,2.616339,twdb010
    """
    try:
        tmp = pd.read_csv(os.path.join (folder, fn))
    except:
        logging.error("read_TWDB_NAM_csv() CRITICAL: Failed to read file {} in folder {} \n exiting the program early!".format(fn, folder))
        return -1
    logging.debug("read_TWDB_NAM_csv() Detected the timestamp from file {} as {} {} {} {} {}"
                  .format(fn,fn[8:12], fn[12:14], fn[14:16],fn[17:19], fn[23:25]))
    #datetime requires integers, create datetime column from the filename and apply deltatime from hour 
    tmp['Datetime'] = datetime.datetime( int(fn[8:12]),int( fn[12:14]), int(fn[14:16]),int(fn[17:19])) +  datetime.timedelta(hours=int(fn[23:25]))
    
    #our our columns winds in U and V components, if so process to mph and dir
    if convertUVwinds:
        #convert U and V to speedmph and dir - converting from mps to mph with constant 2.23694
        tmp['SPD_mph'] = wind_uv_to_spd(tmp[columns_name[0]],tmp[columns_name[1]])* 2.23694
        tmp['DIR'] = wind_uv_to_dir(tmp[columns_name[0]],tmp[columns_name[1]])
        tmp.drop([columns_name[0],columns_name[1]], axis=1, inplace=True)
        
        
    logging.debug("read_TWDB_NAM_csv() Our completed ingested file looks like: \n {}".format(tmp.head(2)))
    
    return tmp




In [5]:

def Convert_TWDB(import_folder,OUTPUT_folder,TMP_folder,windlist):
    """
    Function imports Wind data in csv format, using the filename to create a datetime index. 
    Filename follows the NOAA NAM formatting scheme where the datetime is embedded in the filename.  
    See the NOAA site: https://www.ncei.noaa.gov/products/weather-climate-models/north-american-mesoscale
    """
     #our U and V constants we are expecting from the files
    U = 'UGRD_P0_L103_GLC0'
    V= 'VGRD_P0_L103_GLC0'

    try:
        files = os.listdir(import_folder)  
    except:
        logging.error("We could not open the folder specified {}".format(import_folder))

        
    data = []
    logging.info("Loading wind data from disk: STARTING")
    for i in files: 
        tmp = read_TWDB_NAM_csv(i,import_folder,[U,V],convertUVwinds=True)
        if tmp is None:
            logging.error("Quitting abnormally, see errors from above")
            return
        else:
            data.append(tmp)
    logging.info("Loading wind data from disk: COMPLETED - with {} records".format(len(data)))


    #combine the list of dataframes data[] into one giant frame...
    df = pd.concat(data, axis=0)
    df.sort_values(by=['Datetime'],inplace=True)
    logging.debug("Created one dataframe with record length {} from {} to {}".format(len(df),df['Datetime'].min(),df['Datetime'].max()))
    logging.debug("Our dataframe looks like: \n {} \n {}".format(df.head(2),df.tail(2)))
    
    Print_Winds_TXBLEND_FMT(df,windlist,TMP_folder)
    #add tar.gz operation for output_folder
    
    outputfile = os.path.join(OUTPUT_folder,"twdbq.tar")
    make_tarfile(outputfile, TMP_folder)



In [ ]:


Convert_TWDB('nam_218_20220814_1800',"","twdb-winds-test","NAMwinds.latlist.csv")

INFO:root:Loading wind data from disk: STARTING
DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_000.csv as 2022 08 14 18 00
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station            Datetime    SPD_mph         DIR
0  twdb000 2022-08-14 18:00:00  15.128582  102.088511
1  twdb001 2022-08-14 18:00:00  14.335636  108.621309
DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_001.csv as 2022 08 14 18 01
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station            Datetime    SPD_mph        DIR
0  twdb000 2022-08-14 19:00:00  16.388135  104.79092
1  twdb001 2022-08-14 19:00:00  16.224203  112.02847
DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_002.csv as 2022 08 14 18 02
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station        

DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_024.csv as 2022 08 14 18 24
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station            Datetime    SPD_mph         DIR
0  twdb000 2022-08-15 18:00:00  13.756382  133.989510
1  twdb001 2022-08-15 18:00:00  13.723100  134.515426
DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_025.csv as 2022 08 14 18 25
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station            Datetime    SPD_mph         DIR
0  twdb000 2022-08-15 19:00:00  14.513651  136.874918
1  twdb001 2022-08-15 19:00:00  14.465713  136.818450
DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_026.csv as 2022 08 14 18 26
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station            Datetime    SPD_mph         DIR
0  twdb000 2022-08-15 20:00:00  15.479839  141.434620
1  twdb

DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_072.csv as 2022 08 14 18 72
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station            Datetime   SPD_mph         DIR
0  twdb000 2022-08-17 18:00:00  4.611444  218.401269
1  twdb001 2022-08-17 18:00:00  5.043768  216.796003
DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_075.csv as 2022 08 14 18 75
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station            Datetime   SPD_mph         DIR
0  twdb000 2022-08-17 21:00:00  5.752346  178.017527
1  twdb001 2022-08-17 21:00:00  6.331455  179.008882
DEBUG:root:read_TWDB_NAM_csv() Detected the timestamp from file nam_218_20220814_1800_078.csv as 2022 08 14 18 78
DEBUG:root:read_TWDB_NAM_csv() Our completed ingested file looks like: 
    station   Datetime    SPD_mph         DIR
0  twdb000 2022-08-18  14.843487  166.281238
1  twdb001 2022-08-18  15.33734

DEBUG:root:Writing file twdb-winds-test\twdb216.wndq COMPLETED
DEBUG:root:Processing station twdb233
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
233  twdb233 2022-08-14 18:00:00  11.283383  175.188374
233  twdb233 2022-08-14 19:00:00  13.777872  163.654600
DEBUG:root:Writing file twdb-winds-test\twdb233.wndq COMPLETED
DEBUG:root:Processing station twdb215
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
215  twdb215 2022-08-14 18:00:00   9.657143  174.375685
215  twdb215 2022-08-14 19:00:00  12.530674  163.474285
DEBUG:root:Writing file twdb-winds-test\twdb215.wndq COMPLETED
DEBUG:root:Processing station twdb234
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
234  twdb234 2022-08-14 18:00:00  11.348413  175.329335
234  twdb234 2022-08-14 19:00:00  13.640483  163.289169
DEBUG:root:Writing file twdb-winds-test\

DEBUG:root:Writing file twdb-winds-test\twdb189.wndq COMPLETED
DEBUG:root:Processing station twdb188
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
188  twdb188 2022-08-14 18:00:00   9.025088  174.551297
188  twdb188 2022-08-14 19:00:00  11.264306  164.409321
DEBUG:root:Writing file twdb-winds-test\twdb188.wndq COMPLETED
DEBUG:root:Processing station twdb187
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
187  twdb187 2022-08-14 18:00:00   9.250567  176.213418
187  twdb187 2022-08-14 19:00:00  11.494362  166.685272
DEBUG:root:Writing file twdb-winds-test\twdb187.wndq COMPLETED
DEBUG:root:Processing station twdb186
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
186  twdb186 2022-08-14 18:00:00   9.234602  175.650294
186  twdb186 2022-08-14 19:00:00  11.583146  167.243397
DEBUG:root:Writing file twdb-winds-test\

DEBUG:root:Writing file twdb-winds-test\twdb206.wndq COMPLETED
DEBUG:root:Processing station twdb205
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
205  twdb205 2022-08-14 18:00:00   9.106407  177.281205
205  twdb205 2022-08-14 19:00:00  11.178146  164.286056
DEBUG:root:Writing file twdb-winds-test\twdb205.wndq COMPLETED
DEBUG:root:Processing station twdb204
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
204  twdb204 2022-08-14 18:00:00   9.683819  177.973262
204  twdb204 2022-08-14 19:00:00  11.960073  166.111352
DEBUG:root:Writing file twdb-winds-test\twdb204.wndq COMPLETED
DEBUG:root:Processing station twdb254
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime   SPD_mph         DIR
254  twdb254 2022-08-14 18:00:00  9.923953  192.191791
254  twdb254 2022-08-14 19:00:00  9.858037  170.024755
DEBUG:root:Writing file twdb-winds-test\twd

DEBUG:root:Writing file twdb-winds-test\twdb309.wndq COMPLETED
DEBUG:root:Processing station twdb308
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime   SPD_mph         DIR
308  twdb308 2022-08-14 18:00:00  9.733340  171.766796
308  twdb308 2022-08-14 19:00:00  9.578852  152.558575
DEBUG:root:Writing file twdb-winds-test\twdb308.wndq COMPLETED
DEBUG:root:Processing station twdb307
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
307  twdb307 2022-08-14 18:00:00  10.264407  173.202603
307  twdb307 2022-08-14 19:00:00   9.927977  153.166767
DEBUG:root:Writing file twdb-winds-test\twdb307.wndq COMPLETED
DEBUG:root:Processing station twdb306
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
306  twdb306 2022-08-14 18:00:00  10.939960  175.624685
306  twdb306 2022-08-14 19:00:00  10.248622  154.070015
DEBUG:root:Writing file twdb-winds-test\twd

DEBUG:root:Writing file twdb-winds-test\twdb326.wndq COMPLETED
DEBUG:root:Processing station twdb325
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime   SPD_mph         DIR
325  twdb325 2022-08-14 18:00:00  8.844730  167.242931
325  twdb325 2022-08-14 19:00:00  8.459449  175.550593
DEBUG:root:Writing file twdb-winds-test\twdb325.wndq COMPLETED
DEBUG:root:Processing station twdb324
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime   SPD_mph         DIR
324  twdb324 2022-08-14 18:00:00  8.750927  166.652677
324  twdb324 2022-08-14 19:00:00  8.704544  174.049871
DEBUG:root:Writing file twdb-winds-test\twdb324.wndq COMPLETED
DEBUG:root:Processing station twdb323
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime   SPD_mph         DIR
323  twdb323 2022-08-14 18:00:00  8.864247  166.082766
323  twdb323 2022-08-14 19:00:00  9.185812  172.396058
DEBUG:root:Writing file twdb-winds-test\twdb323.w

DEBUG:root:Writing file twdb-winds-test\twdb277.wndq COMPLETED
DEBUG:root:Processing station twdb278
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime   SPD_mph         DIR
278  twdb278 2022-08-14 18:00:00  9.096221  174.310904
278  twdb278 2022-08-14 19:00:00  7.800737  160.162956
DEBUG:root:Writing file twdb-winds-test\twdb278.wndq COMPLETED
DEBUG:root:Processing station twdb279
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
279  twdb279 2022-08-14 18:00:00  12.141157  165.999439
279  twdb279 2022-08-14 19:00:00  14.279170  155.537001
DEBUG:root:Writing file twdb-winds-test\twdb279.wndq COMPLETED
DEBUG:root:Processing station twdb297
DEBUG:root:Our sliced df has len 53 and looks like      station            Datetime    SPD_mph         DIR
297  twdb297 2022-08-14 18:00:00  10.819349  180.674052
297  twdb297 2022-08-14 19:00:00  10.128419  157.079336
DEBUG:root:Writing file twdb-winds-test\twd